In [1]:
import pandas as pd
df = pd.read_csv('data/ibtracs.since1980.list.v04r00.csv', dtype='object', )

In [12]:
pd.set_option('display.max_columns', None)
df.head(3)

,SID,SEASON,NUMBER,BASIN,SUBBASIN,NAME,ISO_TIME,NATURE,LAT,LON,WMO_WIND,WMO_PRES,WMO_AGENCY,TRACK_TYPE,DIST2LAND,LANDFALL,IFLAG,USA_AGENCY,USA_ATCF_ID,USA_LAT,USA_LON,USA_RECORD,USA_STATUS,USA_WIND,USA_PRES,USA_SSHS,USA_R34_NE,USA_R34_SE,USA_R34_SW,USA_R34_NW,USA_R50_NE,USA_R50_SE,USA_R50_SW,USA_R50_NW,USA_R64_NE,USA_R64_SE,USA_R64_SW,USA_R64_NW,USA_POCI,USA_ROCI,USA_RMW,USA_EYE,TOKYO_LAT,TOKYO_LON,TOKYO_GRADE,TOKYO_WIND,TOKYO_PRES,TOKYO_R50_DIR,TOKYO_R50_LONG,TOKYO_R50_SHORT,TOKYO_R30_DIR,TOKYO_R30_LONG,TOKYO_R30_SHORT,TOKYO_LAND,CMA_LAT,CMA_LON,CMA_CAT,CMA_WIND,CMA_PRES,HKO_LAT,HKO_LON,HKO_CAT,HKO_WIND,HKO_PRES,NEWDELHI_LAT,NEWDELHI_LON,NEWDELHI_GRADE,NEWDELHI_WIND,NEWDELHI_PRES,NEWDELHI_CI,NEWDELHI_DP,NEWDELHI_POCI,REUNION_LAT,REUNION_LON,REUNION_TYPE,REUNION_WIND,REUNION_PRES,REUNION_TNUM,REUNION_CI,REUNION_RMW,REUNION_R34_NE,REUNION_R34_SE,REUNION_R34_SW,REUNION_R34_NW,REUNION_R50_NE,REUNION_R50_SE,REUNION_R50_SW,REUNION_R50_NW,REUNION_R64_NE,REUNION_R64_SE,REUNION_R64_SW,REUNION_R64_NW,BOM_LAT,BOM_LON,BOM_TYPE,BOM_WIND,BOM_PRES,BOM_TNUM,BOM_CI,BOM_RMW,BOM_R34_NE,BOM_R34_SE,BOM_R34_SW,BOM_R34_NW,BOM_R50_NE,BOM_R50_SE,BOM_R50_SW,BOM_R50_NW,BOM_R64_NE,BOM_R64_SE,BOM_R64_SW,BOM_R64_NW,BOM_ROCI,BOM_POCI,BOM_EYE,BOM_POS_METHOD,BOM_PRES_METHOD,NADI_LAT,NADI_LON,NADI_CAT,NADI_WIND,NADI_PRES,WELLINGTON_LAT,WELLINGTON_LON,WELLINGTON_WIND,WELLINGTON_PRES,DS824_LAT,DS824_LON,DS824_STAGE,DS824_WIND,DS824_PRES,TD9636_LAT,TD9636_LON,TD9636_STAGE,TD9636_WIND,TD9636_PRES,TD9635_LAT,TD9635_LON,TD9635_WIND,TD9635_PRES,TD9635_ROCI,NEUMANN_LAT,NEUMANN_LON,NEUMANN_CLASS,NEUMANN_WIND,NEUMANN_PRES,MLC_LAT,MLC_LON,MLC_CLASS,MLC_WIND,MLC_PRES,USA_GUST,BOM_GUST,BOM_GUST_PER,REUNION_GUST,REUNION_GUST_PER,USA_SEAHGT,USA_SEARAD_NE,USA_SEARAD_SE,USA_SEARAD_SW,USA_SEARAD_NW,STORM_SPEED,STORM_DIR
1,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 00:00:00,TS,-12.5000,172.500,,,,main,647,647,O________OO_O_,jtwc_sh,SH051980,-12.5000,172.500,,,25,,-1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-12.5000,172.500,TC,25,,-12.5000,172.500,1,25,,,,,,,-12.5000,172.500,TC,25,,,,,,,,,,,,,,,,,6,351
2,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 03:00:00,TS,-12.1927,172.441,,,,main,653,653,P________PP_P_,,SH051980,-12.1825,172.432,,,25,,-1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-12.1825,172.432,TC,25,,-12.2234,172.469,1,,,,,,,,-12.1825,172.432,TC,25,,,,,,,,,,,,,,,,,6,351
3,1980001S13173,1980,1,SP,MM,PENI,1980-01-01 06:00:00,TS,-11.9144,172.412,,,,main,670,670,O________OP_O_,jtwc_sh,SH051980,-11.9000,172.400,,,25,,-1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,-11.9000,172.400,TC,25,,-11.9575,172.450,1,,,,,,,,-11.9000,172.400,TC,25,,,,,,,,,,,,,,,,,5,358


My data has a multi index so I'm going to drop that in order to read in the columns. I can now see the size of the data.

In [2]:
df.drop(index=0, inplace=True)
df.shape

(271883, 163)

There are 163 columns and they are all reading in as object datatypes. I'll need to go through and clean these up.

In [3]:
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 271883 entries, 1 to 271883
Data columns (total 163 columns):
SID                 object
SEASON              object
NUMBER              object
BASIN               object
SUBBASIN            object
NAME                object
ISO_TIME            object
NATURE              object
LAT                 object
LON                 object
WMO_WIND            object
WMO_PRES            object
WMO_AGENCY          object
TRACK_TYPE          object
DIST2LAND           object
LANDFALL            object
IFLAG               object
USA_AGENCY          object
USA_ATCF_ID         object
USA_LAT             object
USA_LON             object
USA_RECORD          object
USA_STATUS          object
USA_WIND            object
USA_PRES            object
USA_SSHS            object
USA_R34_NE          object
USA_R34_SE          object
USA_R34_SW          object
USA_R34_NW          object
USA_R50_NE          object
USA_R50_SE          object
USA_R50_SW          obje

The dataset has readings for storms at multiple points in their progression. There are 4,458 unique storms tracked.

In [13]:
df.columns = [x.lower() for x in df.columns]

In [3]:
#there are this many separate storms
df['sid'].nunique()

4458

My classification task will be to identify whether they are minor storms or severe Tropical Storms. Looking at my target column, nature, I can see six different classes that I need to clean up. 

NR, not reported, and MX, mixture will be removed as they don't tell me anything. TS, tropical storm, will be my '1' - a severe storm. ET, DS, and SS are extratropical, disturbance, and subtropical - less severe storms. These will be my '0' class. 

In [53]:
df['nature'].unique()

array(['TS', 'ET', 'SS', 'DS'], dtype=object)

In [29]:
df.drop(df.loc[df['nature'] == 'NR'].index, inplace=True)
df.drop(df.loc[df['nature'] == 'MX'].index, inplace=True)

In [61]:
#new column  
df['target'] = 0

# loop through the data and input a 1 where the house was renovated
for row in df.index:
    if df['nature'][row] == 'TS':
        df['target'][row] = 1

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [62]:
df['target'].value_counts(normalize=True)

1    203150
0     23273
Name: target, dtype: int64

So there is a pretty severe class imbalance here. Before I address that, I want to define my features. Taking a preliminary look through the columns in the dataframe and the documentation as a guide, I'm selecting the following as potential features to explore. 

In [64]:
initial_feats = ['sid', 'season', 'number', 'basin', 'subbasin', 'iso_time', 'lat', 'lon', 
                 'wmo_wind', 'dist2land', 'ds824_wind', 'td9636_stage', 'storm_speed', 'storm_dir']

In [65]:
xs_df = df[initial_feats]
xs_df.head()

,sid,season,number,basin,subbasin,iso_time,lat,lon,wmo_wind,dist2land,ds824_wind,td9636_stage,storm_speed,storm_dir
1,1980001S13173,1980,1,SP,MM,1980-01-01 00:00:00,-12.5000,172.500,,647,25,1,6,351
2,1980001S13173,1980,1,SP,MM,1980-01-01 03:00:00,-12.1927,172.441,,653,25,1,6,351
3,1980001S13173,1980,1,SP,MM,1980-01-01 06:00:00,-11.9144,172.412,,670,25,1,5,358
4,1980001S13173,1980,1,SP,MM,1980-01-01 09:00:00,-11.6863,172.435,,682,25,1,4,12
5,1980001S13173,1980,1,SP,MM,1980-01-01 12:00:00,-11.5000,172.500,,703,25,1,4,22


In [66]:
xs_df[['number', 'lat', 'lon', 'dist2land']] = xs_df[['number', 'lat', 'lon', 'dist2land']].apply(pd.to_numeric)
xs_df[['season', 'iso_time']] = xs_df[['season', 'iso_time']].apply(pd.to_datetime)

In [67]:
string_cols = ['sid', 'basin', 'subbasin']

In [70]:
xs_df['ds824_wind'] = xs_df['ds824_wind'].str.replace(' ', '0')

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# xs_df['wmo_wind'] = xs_df['wmo_wind'].str.replace(' ', '0')

In [77]:
remaining_cols = ['wmo_wind', 'ds824_wind', 'td9636_stage', 'storm_speed', 'storm_dir']
xs_df[remaining_cols] = xs_df[remaining_cols].apply(lambda x: x.str.replace(' ', '0'))

In [80]:
xs_df[remaining_cols] = xs_df[remaining_cols].apply(pd.to_numeric)

In [81]:
xs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226423 entries, 1 to 271073
Data columns (total 14 columns):
sid             226423 non-null object
season          226423 non-null datetime64[ns]
number          226423 non-null int64
basin           190696 non-null object
subbasin        197687 non-null object
iso_time        226423 non-null datetime64[ns]
lat             226423 non-null float64
lon             226423 non-null float64
wmo_wind        226423 non-null int64
dist2land       226423 non-null int64
ds824_wind      226423 non-null int64
td9636_stage    226423 non-null int64
storm_speed     226423 non-null int64
storm_dir       226423 non-null int64
dtypes: datetime64[ns](2), float64(2), int64(7), object(3)
memory usage: 35.9+ MB


In [24]:
xs_df['WMO_WIND'] = xs_df['WMO_WIND'].apply(pd.to_numeric)
#use a .max() for groupby 

C:\Users\christina.sarver\Anaconda3\envs\learn-env\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
